In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

import data_loader
import model_lstm 
import model_bilstm_attention 
import evaluate
import utils  # si lo usas para métricas o visualización


import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Cargar datos
df = data_loader.load_data()

# Limpiar texto
df['tweet'] = df['tweet'].apply(utils.clean_text)

# Tokenización
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['tweet'])

sequences = tokenizer.texts_to_sequences(df['tweet'])
padded = pad_sequences(sequences, maxlen=50, padding='post')

# Datos y etiquetas
X = padded
y = np.array(df['label'])

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from tensorflow.keras.optimizers import Adam

vocab_size = 10000
embedding_dim = 64
input_length = 50

model = model_lstm.build_lstm(...)
model = model_bilstm_attention.build_bilstm_attention(...)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=32
)

model.save("../models/rnn_model.h5")


from evaluate import evaluate_model

y_pred = model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype("int32")

evaluate_model(y_test, y_pred_labels)
